In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("drive/MyDrive/Colab Notebooks/CS7650/final")
os.listdir()

Mounted at /content/drive


['data', 'Preprocessing', 'Models', 'resources.gdoc']

In [2]:
import pandas as pd
import json
import numpy as np
import random
import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
class Vocab:
  def __init__(self, w2i_file, wc_file, min_count=1):
    with open(w2i_file, 'r') as file:
      self.word2id = json.load(file)
    with open(wc_file, 'r') as file:
      self.word_counts = json.load(file)
    self.num_words = len(self.word2id.keys())
    self.min_count = min_count
    self.infrequent = [k for k,v in self.word_counts.items() if v <= min_count]
  
  def _word2id(self, word, train):
    if train and (word in self.infrequent and random.random() > 0.5):
      return 0
    else:
      return self.word2id.get(word, 0)

  def sentence2indices(self, sentence, train):
    return [self._word2id(word, train) for word in sentence.split()]
    #return [[self._word2id(word, train) for word in s.split()] for s in sentences]

In [4]:
w2i_file = 'data/vocab/word2id.json'
wc_file = 'data/vocab/word_counts.json'
V = Vocab(w2i_file, wc_file)

In [5]:
df_train = pd.read_csv('data/clean/train_clean.csv')
df_test = pd.read_csv('data/clean/test_clean.csv')
del df_train['Unnamed: 0']
df_train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im really not trying to edit war its j...,0,0,0,0,0,0
3,0001b41b1c6bb37e,more i cant make any real suggestions on impro...,0,0,0,0,0,0
4,0001d958c54c6e35,you sir are my hero any chance you remember wh...,0,0,0,0,0,0


In [6]:
print(len(df_test), len(df_train))

63568 159560


In [7]:
chunks_train = np.array_split(df_train, 4)
for i, df in enumerate(chunks_train):
  filename = 'data/clean_indexed/train_clean_id{0}.csv'.format(i)
  print(filename)
  if not os.path.exists(filename):
    rst = df.copy(deep=True)
    rst['comment_text'] = df['comment_text'].progress_apply(lambda x: V.sentence2indices(x, train=True))
    rst.to_csv(filename, index=False)
    break

data/clean_indexed/train_clean_id0.csv
data/clean_indexed/train_clean_id1.csv
data/clean_indexed/train_clean_id2.csv
data/clean_indexed/train_clean_id3.csv


In [8]:
chunks_test = np.array_split(df_test, 4)
for i, df in enumerate(chunks_test):
  filename = 'data/clean_indexed/test_clean_id{0}.csv'.format(i)
  print(filename)
  if not os.path.exists(filename):
    rst = df.copy(deep=True)
    rst['comment_text'] = df['comment_text'].progress_apply(lambda x: V.sentence2indices(x, train=True))
    rst.to_csv(filename, index=False)
    break

data/clean_indexed/test_clean_id0.csv
data/clean_indexed/test_clean_id1.csv
data/clean_indexed/test_clean_id2.csv
data/clean_indexed/test_clean_id3.csv
